In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/'My Drive'/doan

/content/drive/My Drive/doan


**IMPORT DATA**

In [ ]:
image_size = (64, 64)
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "Class_Human",
    validation_split=0.2,
    subset="training",
    seed=18,
    image_size=image_size,
    batch_size=batch_size,
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "Class_Human",
    validation_split=0.2,
    subset="validation",
    seed=18,
    image_size=image_size,
    batch_size=batch_size,
)


Found 21758 files belonging to 2 classes.
Using 17407 files for training.
Found 21758 files belonging to 2 classes.
Using 4351 files for validation.


In [ ]:
train_ds = train_ds.prefetch(buffer_size=32)
val_ds = val_ds.prefetch(buffer_size=32)

**TRAINNING**

Model

In [ ]:
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten,Rescaling
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from keras.models import Sequential

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger, TensorBoard

In [ ]:
callbacks = [
          #Luu bo trong so weight    
          ModelCheckpoint("files/model.h5"),
          #Theo doi loss function va giam LR
          #factor gia tri learning rate se giam 
          #patience: sau so chu ky ma LR se giam
          ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3),
          #Truyen lai ket qua vao CSV
          CSVLogger("files/data.csv"),
          TensorBoard(),
          #Theo doi loss function
          #patience: sau so chu ky neu khong cai thien thi dung
          #restore_best_weights: lua chon khoi phuc tu epoch co ket qua tot nhat
          EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
]

In [ ]:
input_shape = (64, 64, 3)


model = Sequential()
model.add(Rescaling((1.0 / 255), input_shape=(64, 64, 3)))
model.add(Conv2D(16, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block1'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), name='block1_maxpool'))

model.add(Conv2D(32, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block2'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), name='block2_maxpool'))

model.add(Conv2D(64, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block3'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), name='block3_maxpool'))


model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
def optimizer_init_fn(): 
  learning_rate = 1e-4
  return tf.keras.optimizers.Adam(learning_rate) 

model.compile(optimizer=optimizer_init_fn(),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 64, 64, 3)         0         
                                                                 
 block1 (Conv2D)             (None, 64, 64, 16)        448       
                                                                 
 block1_maxpool (MaxPooling2  (None, 32, 32, 16)       0         
 D)                                                              
                                                                 
 block2 (Conv2D)             (None, 32, 32, 32)        4640      
                                                                 
 block2_maxpool (MaxPooling2  (None, 16, 16, 32)       0         
 D)                                                              
                                                                 
 block3 (Conv2D)             (None, 16, 16, 64)        1

In [ ]:
# Fit model (training)
history = model.fit(train_ds, steps_per_epoch=len(train_ds),
    validation_data=val_ds, validation_steps=len(val_ds), epochs=30, verbose=1,callbacks=callbacks)

Epoch 1/30
544/544 [==============================] - 1561s 3s/step - loss: 0.2715 - accuracy: 0.8903 - val_loss: 0.1399 - val_accuracy: 0.9483 - lr: 1.0000e-04
Epoch 2/30
544/544 [==============================] - 40s 74ms/step - loss: 0.1022 - accuracy: 0.9657 - val_loss: 0.0802 - val_accuracy: 0.9749 - lr: 1.0000e-04
Epoch 3/30
544/544 [==============================] - 41s 75ms/step - loss: 0.0656 - accuracy: 0.9789 - val_loss: 0.0600 - val_accuracy: 0.9793 - lr: 1.0000e-04
Epoch 4/30
544/544 [==============================] - 41s 74ms/step - loss: 0.0452 - accuracy: 0.9855 - val_loss: 0.0537 - val_accuracy: 0.9832 - lr: 1.0000e-04
Epoch 5/30
544/544 [==============================] - 40s 74ms/step - loss: 0.0358 - accuracy: 0.9883 - val_loss: 0.0493 - val_accuracy: 0.9848 - lr: 1.0000e-04
Epoch 6/30
544/544 [==============================] - 41s 75ms/step - loss: 0.0280 - accuracy: 0.9915 - val_loss: 0.0447 - val_accuracy: 0.9853 - lr: 1.0000e-04
Epoch 7/30
544/544 [==============